# Spike LangGraph (Re)Search Flow

In [1]:
%pip install -Uq langchain langgraph langchain_openai langchainhub langsmith duckduckgo-search beautifulsoup4 gradio ipywidgets tavily-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

# Commented out if set in .env file
# os.environ["OPENAI_API_KEY"] = 'lm-studio' # Set to 'lm-studio' for local LMStudio inference
# os.environ["LANGCHAIN_API_KEY"] = ''

# Set LangSmith tracing and project name reference variables
os.environ["LANGCHAIN_TRACING_V2"] = 'true' 
os.environ["LANGCHAIN_PROJECT"] = '20240422-Spike LangGraph Local LLM'

In [3]:
# Import the required libraries
import functools, operator, requests, os, json
from bs4 import BeautifulSoup

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import gradio as gr

/Users/leonvanbokhorst/repos/press-to-talk/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Model

In [4]:
# Initialize model
llm = ChatOpenAI(model="gpt-3.5-turbo")
#llm = ChatOpenAI(model="NA", base_url="http://192.168.50.116:8000/v1") # Use for local inference

### Tools

In [5]:
# Internet Search Tool (TavilySearch)
from langchain_community.tools.tavily_search import TavilySearchResults
internet_search = TavilySearchResults(max_results=3)

# Process Content Tool (BS4)
@tool("process_content", return_direct=False)
def process_content(url: str) -> str:
    """Processes content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

# Tool set
tools = [internet_search, process_content]

# Agents

In [6]:
# Helper function for creating agents
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

# Define agent nodes
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [7]:
# Create Agent Supervisor
members = ["Web_Searcher", "Insight_Researcher"]
system_prompt = (
    "As a supervisor, your role is to oversee a dialogue between these"
    " workers: {members}. Based on the user's request,"
    " determine which worker should take the next action. Each worker is responsible for"
    " executing a specific task and reporting back their findings and progress. Once all tasks are complete,"
    " indicate with 'FINISH'."
)

options = ["FINISH"] + members
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}] }},
        "required": ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
]).partial(options=str(options), members=", ".join(members))

supervisor_chain = (prompt | llm.bind_functions(functions=[function_def], function_call="route") | JsonOutputFunctionsParser())

In [8]:
# Create Search Agent
search_agent = create_agent(llm, tools, "You are a web searcher. Search the internet for information.")
search_node = functools.partial(agent_node, agent=search_agent, name="Web_Searcher")

In [9]:
# Create Web Insight Researcher Agent
insights_research_agent = create_agent(llm, tools, 
        """You are a Insight Researcher. Do step by step. 
        Based on the provided content first identify the list of topics,
        then search internet for each topic one by one
        and finally find insights for each topic one by one.
        Include the insights and sources in the final response
        """)

insights_research_node = functools.partial(agent_node, agent=insights_research_agent, name="Insight_Researcher")

In [10]:
# Define the Agent State
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

workflow = StateGraph(AgentState)
workflow.add_node("Web_Searcher", search_node)
workflow.add_node("Insight_Researcher", insights_research_node)
workflow.add_node("supervisor", supervisor_chain)

In [11]:
# Define edges
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

# Compile the workflow graph
graph = workflow.compile()

In [12]:
# Run the graph
for s in graph.stream({
    "messages": [HumanMessage(content="""Search for the latest veal ear tag technology trends in 2024,
            summarize the content. After summarise pass it on to insight researcher
            to provide insights for each topic""")]
}):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Web_Searcher'}}
----
{'Web_Searcher': {'messages': [HumanMessage(content="I have summarized the content from the sources on the latest veal ear tag technology trends in 2024:\n\n1. **Smart Ear Tags for Livestock Market Insight**:\n   - Market Overview: Smart ear tags utilize RFID technology to collect real-time data about animals' behavior, health, and location.\n   - Market Growth: The market is projected to experience significant growth with a focus on efficiency and IoT adoption.\n   - Market Trends: Cloud-based smart ear tags are preferred for easy data management. Advanced features like temperature monitoring are becoming popular.\n\n2. **Ear Tags in Beef Industry**:\n   - Merck's SenseHub Feedlot technology tracks animal health via ear tags, providing real-time data on temperature and behavior.\n   - Database Reporting: The system helps diagnose illness faster by analyzing abnormal behaviors and changes in animal temperatures.\n\n3. **Electronic Ear Tags 

In [13]:
# final_response = graph.invoke({
#     "messages": [HumanMessage(
#         content="""Search for the latest veal ear tag technology trends in 2024,
#                 summarize the content
#                 and provide insights for each topic.""")]
# })

# print(final_response['messages'][1].content)